In [48]:
library(tximport)
library(ensembldb)
library(AnnotationHub)
library(DESeq2)
library(edgeR)

Loading required package: limma


Attaching package: 'limma'


The following object is masked from 'package:DESeq2':

    plotMA


The following object is masked from 'package:BiocGenerics':

    plotMA




In [2]:
hub = AnnotationHub()

In [9]:
#make sure to use the right species
ensdb_query <- query(hub, c("EnsDb", "sapiens", "113"))
# ensdb_query

AnnotationHub with 1 record
# snapshotDate(): 2024-10-28
# names(): AH119325
# $dataprovider: Ensembl
# $species: Homo sapiens
# $rdataclass: EnsDb
# $rdatadateadded: 2024-10-28
# $title: Ensembl 113 EnsDb for Homo sapiens
# $description: Gene and protein annotations for Homo sapiens based on Ensem...
# $taxonomyid: 9606
# $genome: GRCh38
# $sourcetype: ensembl
# $sourceurl: http://www.ensembl.org
# $sourcesize: NA
# $tags: c("113", "Annotation", "AnnotationHubSoftware", "Coverage",
#   "DataImport", "EnsDb", "Ensembl", "Gene", "Protein", "Sequencing",
#   "Transcript") 
# retrieve record with 'object[["AH119325"]]' 

In [13]:
ensdb <- ensdb_query[['AH119325']]

loading from cache



In [14]:
tx_data <- transcripts(ensdb, return.type = "DataFrame")

In [15]:
tx2gene <- tx_data[, c("tx_id", "gene_id")]

In [36]:
quants_dir <- "Salmon_results_trim/"

quant_files <- list.files(quants_dir, pattern = "quant.sf$", recursive = TRUE, full.names = TRUE)

# quant_dirs <- list.files(quants_dir, pattern = "_GT25", full.names = TRUE)
sample_names <- sub(".*/(.*)/quant.sf$", "\\1", quant_files)

names(quant_files) <- sample_names

quant_files

1021_V1_GT25-02267_CGTATTCGNNNNNNNNN-TGTCGACT_S9 
                  "Salmon_results_trim//1021_V1_GT25-02267_CGTATTCGNNNNNNNNN-TGTCGACT_S9/quant.sf" 
                                          1021_V2__a-IFNa_GT25-02269_TCACAGCANNNNNNNNN-GAGGCATT_S7 
          "Salmon_results_trim//1021_V2__a-IFNa_GT25-02269_TCACAGCANNNNNNNNN-GAGGCATT_S7/quant.sf" 
                                1021_V2__a-IFNg___a-IFNa_GT25-02270_CTCCTAGANNNNNNNNN-GCTTCACA_S13 
"Salmon_results_trim//1021_V2__a-IFNg___a-IFNa_GT25-02270_CTCCTAGANNNNNNNNN-GCTTCACA_S13/quant.sf" 
                                                  1021_V2_GT25-02268_GTTAAGGCNNNNNNNNN-ACCTTCGA_S8 
                  "Salmon_results_trim//1021_V2_GT25-02268_GTTAAGGCNNNNNNNNN-ACCTTCGA_S8/quant.sf" 
                                 1021_V5__a-IFNg___a-IFNa_GT25-02273_ACGACAGANNNNNNNNN-CGCAACTA_S5 
 "Salmon_results_trim//1021_V5__a-IFNg___a-IFNa_GT25-02273_ACGACAGANNNNNNNNN-CGCAACTA_S5/quant.sf" 
                                         1021_V5__a-IFNg_GT25-02272_TGAACCTGNNNNNNNNN-ACCTCTTC_S11 
         "Salmon_results_trim//1021_V5__a-IFNg_GT25-02272_TGAACCTGNNNNNNNNN-ACCTCTTC_S11/quant.sf" 
                                                  1021_V5_GT25-02271_ACACCAGTNNNNNNNNN-AGTGCATC_S3 
                  "Salmon_results_trim//1021_V5_GT25-02271_ACACCAGTNNNNNNNNN-AGTGCATC_S3/quant.sf" 
                                         IFNa___anti-IFNa_GT25-02265_TTCGCAGTNNNNNNNNN-TCCGATCA_S6 
         "Salmon_results_trim//IFNa___anti-IFNa_GT25-02265_TTCGCAGTNNNNNNNNN-TCCGATCA_S6/quant.sf" 
                                                    IFNa_GT25-02264_TCAAGGACNNNNNNNNN-CATTCGTC_S10 
                    "Salmon_results_trim//IFNa_GT25-02264_TCAAGGACNNNNNNNNN-CATTCGTC_S10/quant.sf" 
                                         IFNg___anti-IFNg_GT25-02263_TGGATCACNNNNNNNNN-GCATTGGT_S2 
         "Salmon_results_trim//IFNg___anti-IFNg_GT25-02263_TGGATCACNNNNNNNNN-GCATTGGT_S2/quant.sf" 
                                             IFNg___IFNa_GT25-02266_AGTCGCTTNNNNNNNNN-TCGAACCT_S12 
             "Salmon_results_trim//IFNg___IFNa_GT25-02266_AGTCGCTTNNNNNNNNN-TCGAACCT_S12/quant.sf" 
                                                     IFNg_GT25-02262_CCGGAATTNNNNNNNNN-ACCGAATG_S1 
                     "Salmon_results_trim//IFNg_GT25-02262_CCGGAATTNNNNNNNNN-ACCGAATG_S1/quant.sf" 
                                                   Medium_GT25-02261_CACGTTGTNNNNNNNNN-ACACCTCA_S4 
                   "Salmon_results_trim//Medium_GT25-02261_CACGTTGTNNNNNNNNN-ACACCTCA_S4/quant.sf" 
                                                                                   Undetermined_S0 
                                                   "Salmon_results_trim//Undetermined_S0/quant.sf"

In [37]:
txi <- tximport(quant_files, type = "salmon", tx2gene = tx2gene,ignoreTxVersion = TRUE)

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 


summarizing abundance

summarizing counts

summarizing length



In [42]:
write.csv(txi$counts, file = "gene_level_count.csv", row.names = TRUE)

In [49]:
count_data <- txi$counts 

In [50]:
dge <- DGEList(counts = count_data)

In [52]:
dge <- calcNormFactors(dge, method = "TMM")

In [57]:
cpm_data <- cpm(dge)
keep_genes <- rowSums(cpm_data > 1) >= 2
dge_filtered <- dge[keep_genes, , keep.lib.sizes = FALSE]

In [58]:
normalized_counts <- cpm(dge_filtered, normalized.lib.sizes = TRUE)

In [60]:
write.csv(normalized_counts, file = "gene_level_count_tmm_normalized.csv", row.names = TRUE)